In [13]:
import os
from transformers import pipeline
from deep_translator import GoogleTranslator
import joblib
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import re

# Función para procesar texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [word for word in tokens if len(word) > 2]
    return ' '.join(tokens)

# Configurar autenticación de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="c15dae568e384c639214780a32252f2a",       # Reemplaza con tu Client ID
    client_secret="a05c91e95775407e8a18d34d76b0e73f",  # Reemplaza con tu Client Secret
    redirect_uri="http://localhost:9443/callback",  # URL de redirección configurada
    scope="playlist-modify-public"
))

# Cargar recursos
songs_library = pd.read_csv('backend/data/songs_library.csv')
label_encoder = joblib.load('backend/models/playlist-model/label_encoder.joblib')
scaler = joblib.load('backend/models/playlist-model/scaler.joblib')
playlist_model = joblib.load('backend/models/playlist-model/playlist_model.joblib')
selected_columns = ['track_name', 'artists', 'popularity', 'explicit', 'danceability', 'energy', 'valence', 'track_genre']
numerical_features = ['popularity', 'danceability', 'energy', 'valence']
songs_library= songs_library[selected_columns]



# Procesar canciones
songs_library['track_genre'] = label_encoder.transform(songs_library['track_genre'])
songs_library[numerical_features] = scaler.transform(songs_library[numerical_features])
songs_library['mood'] = playlist_model.predict(songs_library[numerical_features + ['track_genre']])

# Diccionario para traducir emociones del modelo pre-entrenado al español
emotion_translation = {
    "joy": "Alegría",
    "anger": "Enojo",
    "sadness": "Tristeza",
    "fear": "Miedo",
    "surprise": "Sorpresa",
    "love": "Amor",
    "neutral": "Neutral"
}
# Menú principal
print("Introduce el texto para analizar la emoción:")
text = input("Introduce como te sientes hoy: ")

print("\nSelecciona el modelo a usar para detección de emociones:")
print("1. Modelo pre-entrenado")
print("2. Modelo propio")
choice = input("Modelo Pre-enterenado: 1 o  Modelo Propio: 2 ")

if choice == "1":
    translator = GoogleTranslator(source='es', target='en')
    emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
    text_en = translator.translate(text)
    emotion_result = emotion_pipeline([text_en])[0]
    emotion_en = emotion_result['label']
    emotion_es = emotion_translation.get(emotion_en, "otro")
    print(f"Emoción detectada: {emotion_es} (Confianza: {emotion_result['score']:.2f})")
else:
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
    classifier = joblib.load('backend/models/sentiment_model/mood_model.joblib')
    text_processed = preprocess_text(text)
    encoded_input = tokenizer(text_processed, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    with torch.no_grad():
        bert_embeddings = bert_model(**encoded_input).last_hidden_state[:, 0, :].squeeze().numpy()
    emotion_es = classifier.predict([bert_embeddings])[0]
    print(f"Emoción detectada: {emotion_es}")

# Crear playlist
print("\nCreando playlist en Spotify...")
playlist_data = songs_library[songs_library['mood'] == emotion_es].sort_values(by='popularity', ascending=False).drop_duplicates(subset=['track_name', 'artists']).head(50)
user_id = sp.me()["id"]
playlist_name = f"Playlist de {emotion_es}"
spotify_playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True)
track_uris = []

for _, row in playlist_data.iterrows():
    query = f"{row['track_name']} {row['artists']}"
    search_result = sp.search(q=query, type="track", limit=1)
    if search_result['tracks']['items']:
        track_uri = search_result['tracks']['items'][0]['uri']
        track_uris.append(track_uri)

if track_uris:
    sp.playlist_add_items(spotify_playlist["id"], track_uris)
    print(f"Playlist '{playlist_name}' creada exitosamente con {len(track_uris)} canciones.")
else:
    print(f"No se encontraron canciones para el sentimiento: {emotion_es}")


Introduce el texto para analizar la emoción:

Selecciona el modelo a usar para detección de emociones:
1. Modelo pre-entrenado
2. Modelo propio


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Emoción detectada: Alegría (Confianza: 0.56)

Creando playlist en Spotify...
Playlist 'Playlist de Alegría' creada exitosamente con 50 canciones.
